# CISI-IMC demo
Here you can test how CISI-IMC works with a subset of training dataset and evaluation dataset.

# 0. Set up libraries and paths

In [1]:
from pathlib import Path
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Import system libraries to configure code directory as module
from os.path import dirname, abspath, join
import sys

# Find code directory relative to our directory
THIS_DIR = dirname('__file__')
CODE_DIR = abspath(join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

In [3]:
# Import functions from 'code' dir
from smaf import smaf
from utils import analyse_decoding, produce_random_As, sparse_decode_blocks_lasso
from simulate_A import simulate_A

# 1. training U and A
To reduce the calculation time for this demo, training dataset is subset to 10% of original dataset.

In [4]:
# set path
sce_path = '/mnt/projects/data/demo_sces/training_sce_demo.h5ad' # change to where the demo data exists
out_path = Path.cwd()/'output'
out_path.mkdir(parents=True, exist_ok=True)

In [5]:
# read sce
sce = ad.read_h5ad(sce_path)

## 1a. dictionary U 
First, we calculate U (module dictionary) using SMAF on training dataset X. SMAF calculates U by iteratively decomposing X into U and W (single-cell module activity).  

See code/smaf.py for more detailed explanation of each parameter

In [6]:
## Set params for U
# set variables
d = 80            # initial number of modules in U
maxItr = 100      # number of iterations of calculating U and W
nthread = -1      # number of threads used during SMAF

# main parameter
methodW = 'lasso' # lasso or omp to calculate W
nblocksW_lasso =1 # number of blocks into which X was split, during calculating W 
ldaW = 0.02       # error tolerance for lasso for W
ldaU = 0.02       # error torelance for lasso for U

In [7]:
# calculate U
U,W,X = smaf(sce,d,maxItr,methodW,ldaU, ldaW=ldaW, THREADS=nthread,  X_normalization='paper_norm',
          num_blocks_W=nblocksW_lasso, num_blocks_U=1, layer=None, Normalize_U=True, saveItr=False) 

Initialized U, W with NMF, SMAF maxItr =  100


In [8]:
# save U 
U_df = pd.DataFrame(U, columns=list(range(1, U.shape[1]+1)),index=sce.var_names)
U_df.to_csv(os.path.join(out_path, 'U_demo.csv'))

# 1b. barcoding matrix A
Next, to select optimal barcoding marix A, which defines a combination of isotope labels for each target protein, we randomly create As and simulate thier decompression performamce. An A with highest simulated correlation of the worst performing protein is selected as the best A.

In our manuscipt, 8000 As were tested on different subset of training dataset, but here we just simulate 20 As for demonstration purpose.

In [9]:
## set params for random As
# n_A_each * n_L0sum is the total number of As that will be simulated here.
n_A_each = 5     # n of random As produced per unique L0sum
n_L0sum = 4      # n of usinque L0 sum to  test (from the max)

g = np.shape(sce.X)[1] #no of total proteins
m = 8            # n of composite channels 
n = (1,2)        # [min,max] of channels per protein
d_thresh = 0.8   # distance threshold per protein

# Set params for simulatinbg A 
nsr = 0          # one can add random gaussian noise by increasing this (noise-to-signal ratio)
ldaW_dc = 0.002  # error torelance for calculating W during simulation

In [10]:
# priduce random As
Phi = produce_random_As(n_A_each=n_A_each, n_L0sum=n_L0sum, m=m , n=n, d_thresh=d_thresh, g=g)

Random 20 As with L0sum between 29 and 32. 16 genes into 8 channels, max 2 channels per gene
6 4


In [11]:
# simulate random As and record corr for worst performing protein
Mincorr = np.array([])
for phi_id, phi in enumerate(Phi):
    X, Xhat, W, Y = simulate_A(sce, U, phi, nsr, decoding_lasso_lda = ldaW_dc,
                             outpath=None, THREADS=-1, layer=None, num_blocks=20)
    # get simulation results
    res_A, _ = analyse_decoding(phi,U,W,X,Xhat, name="", detail=False)
    Mincorr = np.append(Mincorr,res_A[1])

/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [12]:
# select best A
A = Phi[np.argmax(Mincorr)]

In [13]:
# save A
A_df = pd.DataFrame(A, columns = sce.var.index)
A_df.to_csv(os.path.join(out_path,'A_demo.csv'))

# 2. (Lab experiment) Prepare antibody mix according to the selected A

# 3. (Lab experiment) Image target tissue with the antibody mix to obtain Y
Y: Composite single-cell expression data. To evaluate the decompression performance, co-stain the target tissue with ground truth antibody mix (single channel (e.g., istope label) per antibody). 

# 4. Decompress measured Y
Here we use 10% of the evaluation dataset used in this work, which contains composite data Y and mached ground truth data Xtrue to evaluate the decompression performance. Measured Y is decompressed to A, U, and W, which yeilds decompressed single-cell expression data X via X = UW.

A and U used below is the ones used in the publication, not the ones calculated above, to match the evaluation dataset. 

In [14]:
# set path
sce_path = '/mnt/projects/data/demo_sces/evaluation_sce_demo.h5ad' # change to where the demo data exists
A_path = '/mnt/projects/data/publication_2/9_CISI_actual/A.csv' # change to where the demo data exists
U_path = '/mnt/projects/data/publication_2/9_CISI_actual/U.csv' # change to where the demo data exists

In [15]:
# read sce 
sce = ad.read_h5ad(sce_path)

# subset ground truth channel and CISI channel
sce_gt = sce[:,sce.var.channel_type == "GT"]
sce_cisi = sce[:,sce.var.channel_type == "CISI"]
sce_gt,sce_cisi

(View of AnnData object with n_obs × n_vars = 16140 × 16
     obs: 'sample_id', 'ObjectNumber', 'width_px', 'height_px', 'core_id'
     var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'cellpose', 'channel_type'
     uns: 'X_name',
 View of AnnData object with n_obs × n_vars = 16140 × 8
     obs: 'sample_id', 'ObjectNumber', 'width_px', 'height_px', 'core_id'
     var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'cellpose', 'channel_type'
     uns: 'X_name')

In [16]:
# read A and U
A = pd.read_csv(A_path, index_col = 0)
U = pd.read_csv(U_path, index_col = 0)

## Decompress and evaluate

In [17]:
# parameters
ldaW_dc = 0.02
nblocks = 200

# get composite data and ground truth data from anndata
Y = (sce_cisi.X).T
Xtrue = (sce_gt.X).T

# Decode
W = sparse_decode_blocks_lasso(Y = Y, D = np.array(A).dot(np.array(U)), lda = ldaW_dc, numThreads = -1, num_blocks = nblocks)
Xhat = np.array(U).dot(W)
Xhat[np.isnan(Xhat)] = 0

# analyze results
res, coln, details = analyse_decoding(A,U,W,Xtrue, Xhat, name="", detail=True)

# general results
df = pd.DataFrame(res).T # .T added since it's 1D DF
df.columns = coln

# genewise results
df_genewise = pd.DataFrame(details[0]).T
df_genewise.columns = [p.split('_')[-1] for p in sce_gt.var.index]

/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [18]:
# summary of results
df

,Gene_average,Gene_minimum,Cell_average,Fit,W_l0_mean,AU_90p_coherence
0,0.790004,0.515052,0.708447,0.468801,4.96487,0.637316


In [19]:
# gene-wise Pearson's correlation between decompressed expression and ground truth expression
df_genewise

,MPO,SMA,CD38,CD68,CD163,CD20,CD3,DC-LAMP,CD7,GranzymeB,Ki-67,CD8a,FOXP3,CD303,CD4,CD31
0,0.7063,0.884766,0.878443,0.659911,0.831331,0.928919,0.897238,0.62288,0.900949,0.848323,0.685,0.84122,0.515052,0.691687,0.950289,0.797758
